In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# ====== 1. Generate Dataset ======
np.random.seed(42)
n_samples = 5000

data = {
    'age': np.random.randint(18, 80, n_samples),
    'credit_score': np.random.normal(650, 100, n_samples).astype(int),
    'balance': np.round(np.random.exponential(50000, n_samples), 2),
    'products_number': np.random.randint(1, 5, n_samples),
    'estimated_salary': np.round(np.random.uniform(20000, 150000, n_samples), 2),
    'country': np.random.choice(['Germany', 'France', 'Spain'], n_samples),
    'gender': np.random.choice(['Male', 'Female'], n_samples),
    'tenure': np.random.randint(0, 10, n_samples),
    'has_credit_card': np.random.choice([0, 1], n_samples),
    'is_active_member': np.random.choice([0, 1], n_samples)
}

df = pd.DataFrame(data)

# Create target variable
df['churn'] = (
    (df['balance'] > 80000) | 
    (df['credit_score'] < 550) |
    (df['products_number'] == 1) |
    ((df['age'] > 65) & (df['is_active_member'] == 0))
).astype(int)

print(f"Dataset: {len(df)} rows × {len(df.columns)} columns")
print(f"Churn rate: {df['churn'].mean():.2%}\n")

# ====== 2. Data Preprocessing ======
# Encode categorical variables
le_country = LabelEncoder()
le_gender = LabelEncoder()
df['country'] = le_country.fit_transform(df['country'])
df['gender'] = le_gender.fit_transform(df['gender'])

# Separate features and target
X = df.drop('churn', axis=1).values
y = df['churn'].values.reshape(-1, 1)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}\n")

# ====== 3. Build 3-Layer MLP with NumPy ======
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Xavier initialization for better convergence
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output, learning_rate):
        m = X.shape[0]
        
        # Calculate gradient for output layer
        dz2 = output - y
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Calculate gradient for hidden layer
        dz1 = np.dot(dz2, self.W2.T) * self.sigmoid_derivative(self.a1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Update weights and biases
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def compute_loss(self, y_true, y_pred):
        loss = -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))
        return loss
    
    def train(self, X, y, epochs, learning_rate):
        losses = []
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            loss = self.compute_loss(y, output)
            losses.append(loss)
            
            if (epoch + 1) % 200 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        output = self.forward(X)
        return (output > 0.5).astype(int)

# ====== 4. Train Model ======
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1

mlp = MLP(input_size, hidden_size, output_size)
print("Training MLP with improved settings...\n")
losses = mlp.train(X_train, y_train, epochs=1000, learning_rate=0.1)

# ====== 5. Evaluation ======
y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)

train_accuracy = np.mean(y_pred_train == y_train)
test_accuracy = np.mean(y_pred_test == y_test)

print(f"\n✓ Train Accuracy: {train_accuracy:.4f}")
print(f"✓ Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# ====== Question 3: Compare Activation Functions ======

class MLP_MultiActivation:
    def __init__(self, input_size, hidden_size, output_size, activation='sigmoid'):
        self.activation_type = activation
        
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return (z > 0).astype(float)
    
    def tanh(self, z):
        return np.tanh(z)
    
    def tanh_derivative(self, z):
        return 1 - z**2
    
    def activate(self, z):
        if self.activation_type == 'sigmoid':
            return self.sigmoid(z)
        elif self.activation_type == 'relu':
            return self.relu(z)
        elif self.activation_type == 'tanh':
            return self.tanh(z)
    
    def activate_derivative(self, z):
        if self.activation_type == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif self.activation_type == 'relu':
            return self.relu_derivative(z)
        elif self.activation_type == 'tanh':
            return self.tanh_derivative(z)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.activate(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)  # Output layer always sigmoid for binary classification
        return self.a2
    
    def backward(self, X, y, output, learning_rate):
        m = X.shape[0]
        
        # Output layer gradient
        dz2 = output - y
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Hidden layer gradient
        dz1 = np.dot(dz2, self.W2.T) * self.activate_derivative(self.a1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Update weights
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def compute_loss(self, y_true, y_pred):
        loss = -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))
        return loss
    
    def train(self, X, y, epochs, learning_rate, verbose=False):
        losses = []
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            loss = self.compute_loss(y, output)
            losses.append(loss)
            
            if verbose and (epoch + 1) % 200 == 0:
                print(f"  Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        output = self.forward(X)
        return (output > 0.5).astype(int)

# Train and compare different activation functions
activations = ['sigmoid', 'relu', 'tanh']
results = {}

print("=" * 60)
print("Comparing Activation Functions: Sigmoid vs ReLU vs Tanh")
print("=" * 60)

for activation in activations:
    print(f"\n--- Training with {activation.upper()} activation ---")
    
    model = MLP_MultiActivation(input_size, hidden_size, output_size, activation=activation)
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.1, verbose=True)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    results[activation] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'final_loss': losses[-1],
        'losses': losses
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")

# Summary comparison
print("\n" + "=" * 60)
print("COMPARISON SUMMARY")
print("=" * 60)
print(f"{'Activation':<12} {'Train Acc':<12} {'Test Acc':<12} {'Final Loss':<12}")
print("-" * 60)
for activation in activations:
    r = results[activation]
    print(f"{activation.upper():<12} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f} {r['final_loss']:<12.4f}")

# Find best activation
best_activation = max(results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\n🏆 Best Activation: {best_activation[0].upper()} with Test Accuracy = {best_activation[1]['test_accuracy']:.4f}")

In [ ]:
# ====== Question 4: Experiment with Learning Rates ======
import matplotlib.pyplot as plt

# Test different learning rates
learning_rates = [0.001, 0.01, 0.1]
lr_results = {}

print("=" * 60)
print("Experimenting with Different Learning Rates")
print("=" * 60)

for lr in learning_rates:
    print(f"\n--- Training with Learning Rate = {lr} ---")
    
    model = MLP(input_size, hidden_size, output_size)
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=lr)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    lr_results[lr] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'losses': losses
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")
    print(f"✓ Final Loss: {losses[-1]:.4f}")

# Visualize loss curves
print("\n--- Visualizing Convergence Speed ---")
plt.figure(figsize=(12, 5))

# Plot 1: Loss curves
plt.subplot(1, 2, 1)
for lr in learning_rates:
    plt.plot(lr_results[lr]['losses'], label=f'LR = {lr}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves for Different Learning Rates')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Accuracy comparison
plt.subplot(1, 2, 2)
lrs = [str(lr) for lr in learning_rates]
train_accs = [lr_results[lr]['train_accuracy'] for lr in learning_rates]
test_accs = [lr_results[lr]['test_accuracy'] for lr in learning_rates]

x = np.arange(len(lrs))
width = 0.35

plt.bar(x - width/2, train_accs, width, label='Train Accuracy', alpha=0.8)
plt.bar(x + width/2, test_accs, width, label='Test Accuracy', alpha=0.8)
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.xticks(x, lrs)
plt.legend()
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 60)
print("LEARNING RATE COMPARISON SUMMARY")
print("=" * 60)
print(f"{'LR':<10} {'Train Acc':<12} {'Test Acc':<12} {'Final Loss':<12}")
print("-" * 60)
for lr in learning_rates:
    r = lr_results[lr]
    print(f"{lr:<10} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f} {r['losses'][-1]:<12.4f}")

# Find best learning rate
best_lr = max(lr_results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\n🏆 Best Learning Rate: {best_lr[0]} with Test Accuracy = {best_lr[1]['test_accuracy']:.4f}")

In [ ]:
# ====== Question 5: Analyze Hidden Layer Size Impact ======

# Test different hidden layer sizes
hidden_sizes = [16, 64, 128]
hidden_results = {}

print("=" * 60)
print("Analyzing Hidden Layer Size Impact")
print("=" * 60)

for h_size in hidden_sizes:
    print(f"\n--- Training with Hidden Size = {h_size} neurons ---")
    
    model = MLP(input_size, h_size, output_size)
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.1)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    # Count parameters
    total_params = (input_size * h_size + h_size) + (h_size * output_size + output_size)
    
    hidden_results[h_size] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'losses': losses,
        'parameters': total_params
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")
    print(f"✓ Total Parameters: {total_params}")

# Visualize results
print("\n--- Visualizing Hidden Layer Impact ---")
plt.figure(figsize=(14, 5))

# Plot 1: Loss curves
plt.subplot(1, 3, 1)
for h_size in hidden_sizes:
    plt.plot(hidden_results[h_size]['losses'], label=f'{h_size} neurons')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves for Different Hidden Sizes')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Accuracy comparison
plt.subplot(1, 3, 2)
sizes = [str(h) for h in hidden_sizes]
train_accs = [hidden_results[h]['train_accuracy'] for h in hidden_sizes]
test_accs = [hidden_results[h]['test_accuracy'] for h in hidden_sizes]

x = np.arange(len(sizes))
width = 0.35

plt.bar(x - width/2, train_accs, width, label='Train Accuracy', alpha=0.8)
plt.bar(x + width/2, test_accs, width, label='Test Accuracy', alpha=0.8)
plt.xlabel('Hidden Layer Size')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.xticks(x, sizes)
plt.legend()
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

# Plot 3: Parameters vs Accuracy
plt.subplot(1, 3, 3)
params = [hidden_results[h]['parameters'] for h in hidden_sizes]
test_accs = [hidden_results[h]['test_accuracy'] for h in hidden_sizes]

plt.plot(params, test_accs, marker='o', linewidth=2, markersize=10)
for i, h in enumerate(hidden_sizes):
    plt.annotate(f'{h} neurons', (params[i], test_accs[i]), 
                textcoords="offset points", xytext=(0,10), ha='center')
plt.xlabel('Number of Parameters')
plt.ylabel('Test Accuracy')
plt.title('Model Complexity vs Performance')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 70)
print("HIDDEN LAYER SIZE COMPARISON SUMMARY")
print("=" * 70)
print(f"{'Hidden Size':<15} {'Parameters':<15} {'Train Acc':<12} {'Test Acc':<12}")
print("-" * 70)
for h_size in hidden_sizes:
    r = hidden_results[h_size]
    print(f"{h_size:<15} {r['parameters']:<15} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f}")

# Find best hidden size
best_hidden = max(hidden_results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\n🏆 Best Hidden Size: {best_hidden[0]} neurons with Test Accuracy = {best_hidden[1]['test_accuracy']:.4f}")

# Analysis
print("\n" + "=" * 70)
print("ANALYSIS")
print("=" * 70)
overfitting_gap = []
for h_size in hidden_sizes:
    gap = hidden_results[h_size]['train_accuracy'] - hidden_results[h_size]['test_accuracy']
    overfitting_gap.append(gap)
    status = "Possible Overfitting" if gap > 0.05 else "Good Generalization"
    print(f"Hidden Size {h_size}: Train-Test Gap = {gap:.4f} → {status}")

In [ ]:
# ====== Question 6: Add Regularization Techniques (L2 & Dropout) ======

class MLP_Regularized:
    def __init__(self, input_size, hidden_size, output_size, l2_lambda=0.0, dropout_rate=0.0):
        self.l2_lambda = l2_lambda
        self.dropout_rate = dropout_rate
        
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def dropout(self, X, training=True):
        if training and self.dropout_rate > 0:
            mask = np.random.binomial(1, 1-self.dropout_rate, size=X.shape) / (1-self.dropout_rate)
            return X * mask, mask
        return X, None
    
    def forward(self, X, training=True):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        
        # Apply dropout to hidden layer
        self.a1_dropout, self.dropout_mask = self.dropout(self.a1, training)
        
        self.z2 = np.dot(self.a1_dropout, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output, learning_rate):
        m = X.shape[0]
        
        # Output layer gradient
        dz2 = output - y
        dW2 = np.dot(self.a1_dropout.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Add L2 regularization to weights
        if self.l2_lambda > 0:
            dW2 += (self.l2_lambda / m) * self.W2
        
        # Hidden layer gradient
        dz1 = np.dot(dz2, self.W2.T) * self.sigmoid_derivative(self.a1_dropout)
        
        # Apply dropout mask during backprop
        if self.dropout_mask is not None:
            dz1 = dz1 * self.dropout_mask
        
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Add L2 regularization to weights
        if self.l2_lambda > 0:
            dW1 += (self.l2_lambda / m) * self.W1
        
        # Update weights
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        cross_entropy_loss = -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))
        
        # Add L2 regularization term to loss
        if self.l2_lambda > 0:
            l2_loss = (self.l2_lambda / (2 * m)) * (np.sum(self.W1**2) + np.sum(self.W2**2))
            return cross_entropy_loss + l2_loss
        
        return cross_entropy_loss
    
    def train(self, X, y, epochs, learning_rate, verbose=False):
        losses = []
        for epoch in range(epochs):
            output = self.forward(X, training=True)
            self.backward(X, y, output, learning_rate)
            loss = self.compute_loss(y, output)
            losses.append(loss)
            
            if verbose and (epoch + 1) % 200 == 0:
                print(f"  Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        output = self.forward(X, training=False)
        return (output > 0.5).astype(int)

# Test different regularization techniques
configs = [
    {'name': 'No Regularization', 'l2': 0.0, 'dropout': 0.0},
    {'name': 'L2 Only (λ=0.01)', 'l2': 0.01, 'dropout': 0.0},
    {'name': 'Dropout Only (p=0.2)', 'l2': 0.0, 'dropout': 0.2},
    {'name': 'L2 + Dropout', 'l2': 0.01, 'dropout': 0.2}
]

reg_results = {}

print("=" * 60)
print("Testing Regularization Techniques")
print("=" * 60)

for config in configs:
    print(f"\n--- {config['name']} ---")
    
    model = MLP_Regularized(input_size, hidden_size, output_size, 
                           l2_lambda=config['l2'], dropout_rate=config['dropout'])
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.1, verbose=True)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    reg_results[config['name']] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'losses': losses,
        'overfitting_gap': train_acc - test_acc
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")
    print(f"✓ Overfitting Gap: {train_acc - test_acc:.4f}")

# Visualize results
print("\n--- Visualizing Regularization Impact ---")
plt.figure(figsize=(14, 5))

# Plot 1: Loss curves
plt.subplot(1, 3, 1)
for name in reg_results.keys():
    plt.plot(reg_results[name]['losses'], label=name)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves with Different Regularization')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Accuracy comparison
plt.subplot(1, 3, 2)
names = list(reg_results.keys())
train_accs = [reg_results[n]['train_accuracy'] for n in names]
test_accs = [reg_results[n]['test_accuracy'] for n in names]

x = np.arange(len(names))
width = 0.35

plt.bar(x - width/2, train_accs, width, label='Train Accuracy', alpha=0.8)
plt.bar(x + width/2, test_accs, width, label='Test Accuracy', alpha=0.8)
plt.xlabel('Regularization Method')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.xticks(x, names, rotation=15, ha='right')
plt.legend()
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

# Plot 3: Overfitting gap
plt.subplot(1, 3, 3)
gaps = [reg_results[n]['overfitting_gap'] for n in names]
colors = ['red' if g > 0.05 else 'green' for g in gaps]

plt.bar(names, gaps, color=colors, alpha=0.7)
plt.xlabel('Regularization Method')
plt.ylabel('Train-Test Gap')
plt.title('Overfitting Analysis')
plt.xticks(rotation=15, ha='right')
plt.axhline(y=0.05, color='orange', linestyle='--', label='Threshold (0.05)')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 80)
print("REGULARIZATION COMPARISON SUMMARY")
print("=" * 80)
print(f"{'Method':<25} {'Train Acc':<12} {'Test Acc':<12} {'Gap':<12}")
print("-" * 80)
for name in reg_results.keys():
    r = reg_results[name]
    print(f"{name:<25} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f} {r['overfitting_gap']:<12.4f}")

# Find best method
best_method = min(reg_results.items(), key=lambda x: x[1]['overfitting_gap'])
print(f"\n🏆 Best Method (lowest overfitting): {best_method[0]} with Gap = {best_method[1]['overfitting_gap']:.4f}")

best_test_acc = max(reg_results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"🎯 Best Test Accuracy: {best_test_acc[0]} with Accuracy = {best_test_acc[1]['test_accuracy']:.4f}")

In [ ]:
# ====== Question 7: Comprehensive Evaluation Metrics ======
from sklearn.metrics import confusion_matrix, roc_curve, auc

def calculate_metrics(y_true, y_pred, y_pred_proba):
    """
    Calculate comprehensive evaluation metrics
    """
    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # Precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    # Recall (Sensitivity)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Specificity
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # ROC-AUC
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'specificity': specificity,
        'roc_auc': roc_auc,
        'confusion_matrix': (tn, fp, fn, tp),
        'roc_curve': (fpr, tpr)
    }

# Train a model for evaluation
print("=" * 60)
print("Comprehensive Evaluation Metrics")
print("=" * 60)

print("\n--- Training Model ---")
model = MLP_Regularized(input_size, hidden_size, output_size, l2_lambda=0.01, dropout_rate=0.2)
losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.1, verbose=True)

# Get predictions and probabilities
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

y_pred_proba_train = model.forward(X_train, training=False)
y_pred_proba_test = model.forward(X_test, training=False)

# Calculate metrics for train and test sets
train_metrics = calculate_metrics(y_train, y_pred_train, y_pred_proba_train)
test_metrics = calculate_metrics(y_test, y_pred_test, y_pred_proba_test)

# Display metrics
print("\n" + "=" * 60)
print("TRAIN SET METRICS")
print("=" * 60)
print(f"Accuracy:    {train_metrics['accuracy']:.4f}")
print(f"Precision:   {train_metrics['precision']:.4f}")
print(f"Recall:      {train_metrics['recall']:.4f}")
print(f"F1-Score:    {train_metrics['f1_score']:.4f}")
print(f"Specificity: {train_metrics['specificity']:.4f}")
print(f"ROC-AUC:     {train_metrics['roc_auc']:.4f}")

print("\n" + "=" * 60)
print("TEST SET METRICS")
print("=" * 60)
print(f"Accuracy:    {test_metrics['accuracy']:.4f}")
print(f"Precision:   {test_metrics['precision']:.4f}")
print(f"Recall:      {test_metrics['recall']:.4f}")
print(f"F1-Score:    {test_metrics['f1_score']:.4f}")
print(f"Specificity: {test_metrics['specificity']:.4f}")
print(f"ROC-AUC:     {test_metrics['roc_auc']:.4f}")

# Visualize metrics
print("\n--- Visualizing Evaluation Metrics ---")
plt.figure(figsize=(15, 10))

# Plot 1: Confusion Matrix - Train
plt.subplot(2, 3, 1)
tn, fp, fn, tp = train_metrics['confusion_matrix']
cm_train = np.array([[tn, fp], [fn, tp]])
plt.imshow(cm_train, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Train')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['No Churn', 'Churn'])
plt.yticks(tick_marks, ['No Churn', 'Churn'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm_train[i, j], ha="center", va="center", color="red", fontsize=16)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Plot 2: Confusion Matrix - Test
plt.subplot(2, 3, 2)
tn, fp, fn, tp = test_metrics['confusion_matrix']
cm_test = np.array([[tn, fp], [fn, tp]])
plt.imshow(cm_test, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Test')
plt.colorbar()
plt.xticks(tick_marks, ['No Churn', 'Churn'])
plt.yticks(tick_marks, ['No Churn', 'Churn'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm_test[i, j], ha="center", va="center", color="red", fontsize=16)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Plot 3: Metrics Comparison
plt.subplot(2, 3, 3)
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Specificity']
train_values = [train_metrics['accuracy'], train_metrics['precision'], 
                train_metrics['recall'], train_metrics['f1_score'], train_metrics['specificity']]
test_values = [test_metrics['accuracy'], test_metrics['precision'], 
               test_metrics['recall'], test_metrics['f1_score'], test_metrics['specificity']]

x = np.arange(len(metrics_names))
width = 0.35

plt.bar(x - width/2, train_values, width, label='Train', alpha=0.8)
plt.bar(x + width/2, test_values, width, label='Test', alpha=0.8)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Performance Metrics Comparison')
plt.xticks(x, metrics_names, rotation=45, ha='right')
plt.legend()
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

# Plot 4: ROC Curve - Train
plt.subplot(2, 3, 4)
fpr_train, tpr_train = train_metrics['roc_curve']
plt.plot(fpr_train, tpr_train, color='blue', lw=2, 
         label=f'ROC Curve (AUC = {train_metrics["roc_auc"]:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Train Set')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# Plot 5: ROC Curve - Test
plt.subplot(2, 3, 5)
fpr_test, tpr_test = test_metrics['roc_curve']
plt.plot(fpr_test, tpr_test, color='green', lw=2, 
         label=f'ROC Curve (AUC = {test_metrics["roc_auc"]:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Test Set')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# Plot 6: Metrics Radar Chart
plt.subplot(2, 3, 6, projection='polar')
categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Specificity']
values_test = [test_metrics['accuracy'], test_metrics['precision'], 
               test_metrics['recall'], test_metrics['f1_score'], test_metrics['specificity']]
values_test += values_test[:1]  # Complete the circle

angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

ax = plt.gca()
ax.plot(angles, values_test, 'o-', linewidth=2, label='Test Set')
ax.fill(angles, values_test, alpha=0.25)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.set_title('Performance Radar Chart', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

# Summary
print("\n" + "=" * 60)
print("EVALUATION SUMMARY")
print("=" * 60)
print(f"✓ The model achieves {test_metrics['f1_score']:.2%} F1-Score on test set")
print(f"✓ ROC-AUC of {test_metrics['roc_auc']:.4f} indicates {'excellent' if test_metrics['roc_auc'] > 0.9 else 'good' if test_metrics['roc_auc'] > 0.8 else 'fair'} discrimination")
print(f"✓ Recall: {test_metrics['recall']:.2%} (ability to find churned customers)")
print(f"✓ Precision: {test_metrics['precision']:.2%} (accuracy of churn predictions)")

In [ ]:
# ====== Question 8: Visualize Training Progress ======

class MLP_WithValidation:
    def __init__(self, input_size, hidden_size, output_size, l2_lambda=0.0, dropout_rate=0.0):
        self.l2_lambda = l2_lambda
        self.dropout_rate = dropout_rate
        
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def dropout(self, X, training=True):
        if training and self.dropout_rate > 0:
            mask = np.random.binomial(1, 1-self.dropout_rate, size=X.shape) / (1-self.dropout_rate)
            return X * mask, mask
        return X, None
    
    def forward(self, X, training=True):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        
        self.a1_dropout, self.dropout_mask = self.dropout(self.a1, training)
        
        self.z2 = np.dot(self.a1_dropout, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output, learning_rate):
        m = X.shape[0]
        
        dz2 = output - y
        dW2 = np.dot(self.a1_dropout.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        if self.l2_lambda > 0:
            dW2 += (self.l2_lambda / m) * self.W2
        
        dz1 = np.dot(dz2, self.W2.T) * self.sigmoid_derivative(self.a1_dropout)
        
        if self.dropout_mask is not None:
            dz1 = dz1 * self.dropout_mask
        
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        if self.l2_lambda > 0:
            dW1 += (self.l2_lambda / m) * self.W1
        
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        cross_entropy_loss = -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))
        
        if self.l2_lambda > 0:
            l2_loss = (self.l2_lambda / (2 * m)) * (np.sum(self.W1**2) + np.sum(self.W2**2))
            return cross_entropy_loss + l2_loss
        
        return cross_entropy_loss
    
    def compute_accuracy(self, y_true, y_pred):
        predictions = (y_pred > 0.5).astype(int)
        return np.mean(predictions == y_true)
    
    def train(self, X_train, y_train, X_val, y_val, epochs, learning_rate):
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        
        for epoch in range(epochs):
            # Training phase
            output_train = self.forward(X_train, training=True)
            self.backward(X_train, y_train, output_train, learning_rate)
            train_loss = self.compute_loss(y_train, output_train)
            train_acc = self.compute_accuracy(y_train, output_train)
            
            # Validation phase (no dropout)
            output_val = self.forward(X_val, training=False)
            val_loss = self.compute_loss(y_val, output_val)
            val_acc = self.compute_accuracy(y_val, output_val)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)
            
            if (epoch + 1) % 200 == 0:
                print(f"Epoch {epoch+1}/{epochs}")
                print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        return {
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accuracies': train_accuracies,
            'val_accuracies': val_accuracies
        }
    
    def predict(self, X):
        output = self.forward(X, training=False)
        return (output > 0.5).astype(int)

# Split train data into train and validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print("=" * 60)
print("Training Progress Visualization")
print("=" * 60)
print(f"Train set: {X_train_split.shape[0]} samples")
print(f"Validation set: {X_val_split.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples\n")

# Train model with validation tracking
print("--- Training Model ---")
model = MLP_WithValidation(input_size, hidden_size, output_size, l2_lambda=0.01, dropout_rate=0.2)
history = model.train(X_train_split, y_train_split, X_val_split, y_val_split, 
                     epochs=1000, learning_rate=0.1)

# Final evaluation on test set
y_pred_test = model.predict(X_test)
test_accuracy = np.mean(y_pred_test == y_test)
print(f"\n✓ Final Test Accuracy: {test_accuracy:.4f}")

# Visualize training progress
print("\n--- Visualizing Training Progress ---")
plt.figure(figsize=(16, 10))

# Plot 1: Loss curves
plt.subplot(2, 3, 1)
plt.plot(history['train_losses'], label='Train Loss', linewidth=2)
plt.plot(history['val_losses'], label='Validation Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Accuracy curves
plt.subplot(2, 3, 2)
plt.plot(history['train_accuracies'], label='Train Accuracy', linewidth=2)
plt.plot(history['val_accuracies'], label='Validation Accuracy', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Loss difference (overfitting indicator)
plt.subplot(2, 3, 3)
loss_diff = np.array(history['val_losses']) - np.array(history['train_losses'])
plt.plot(loss_diff, color='red', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Validation Loss - Train Loss')
plt.title('Overfitting Indicator (Loss Gap)')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)

# Plot 4: Smoothed loss curves
plt.subplot(2, 3, 4)
window = 50
if len(history['train_losses']) >= window:
    train_loss_smooth = np.convolve(history['train_losses'], np.ones(window)/window, mode='valid')
    val_loss_smooth = np.convolve(history['val_losses'], np.ones(window)/window, mode='valid')
    plt.plot(train_loss_smooth, label='Train Loss (Smoothed)', linewidth=2)
    plt.plot(val_loss_smooth, label='Val Loss (Smoothed)', linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Smoothed Loss Curves (Window={window})')
    plt.legend()
    plt.grid(True, alpha=0.3)

# Plot 5: Accuracy difference
plt.subplot(2, 3, 5)
acc_diff = np.array(history['train_accuracies']) - np.array(history['val_accuracies'])
plt.plot(acc_diff, color='orange', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Train Acc - Val Acc')
plt.title('Generalization Gap (Accuracy)')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
plt.axhline(y=0.05, color='red', linestyle='--', alpha=0.3, label='Warning Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 6: Learning rate schedule (if needed)
plt.subplot(2, 3, 6)
epochs_range = np.arange(len(history['train_losses']))
final_train_loss = history['train_losses'][-1]
final_val_loss = history['val_losses'][-1]
final_train_acc = history['train_accuracies'][-1]
final_val_acc = history['val_accuracies'][-1]

stats_text = f"""
Final Statistics:

Train Loss: {final_train_loss:.4f}
Val Loss: {final_val_loss:.4f}

Train Acc: {final_train_acc:.4f}
Val Acc: {final_val_acc:.4f}

Test Acc: {test_accuracy:.4f}

Loss Gap: {final_val_loss - final_train_loss:.4f}
Acc Gap: {final_train_acc - final_val_acc:.4f}
"""

plt.text(0.5, 0.5, stats_text, transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='center', horizontalalignment='center',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
         family='monospace')
plt.axis('off')
plt.title('Training Summary Statistics')

plt.tight_layout()
plt.show()

# Analysis
print("\n" + "=" * 60)
print("TRAINING PROGRESS ANALYSIS")
print("=" * 60)

# Check for overfitting
final_loss_gap = history['val_losses'][-1] - history['train_losses'][-1]
final_acc_gap = history['train_accuracies'][-1] - history['val_accuracies'][-1]

print(f"Final Training Loss: {history['train_losses'][-1]:.4f}")
print(f"Final Validation Loss: {history['val_losses'][-1]:.4f}")
print(f"Loss Gap: {final_loss_gap:.4f}")

print(f"\nFinal Training Accuracy: {history['train_accuracies'][-1]:.4f}")
print(f"Final Validation Accuracy: {history['val_accuracies'][-1]:.4f}")
print(f"Accuracy Gap: {final_acc_gap:.4f}")

# Convergence check
if abs(history['train_losses'][-1] - history['train_losses'][-100]) < 0.001:
    print("\n✓ Model has converged")
else:
    print("\n⚠ Model may benefit from more epochs")

# Overfitting check
if final_acc_gap > 0.05:
    print("⚠ Warning: Possible overfitting detected!")
    print("  Consider: more regularization, more data, or early stopping")
else:
    print("✓ Good generalization - no significant overfitting")

In [ ]:
# ====== Question 9: Feature Importance Analysis ======

def permutation_importance(model, X, y, feature_names, n_repeats=10):
    """
    Calculate feature importance using permutation method
    """
    # Baseline accuracy
    baseline_predictions = model.predict(X)
    baseline_accuracy = np.mean(baseline_predictions == y)
    
    importances = []
    
    for i, feature_name in enumerate(feature_names):
        print(f"  Calculating importance for: {feature_name}")
        
        feature_importances = []
        for _ in range(n_repeats):
            # Copy data and shuffle one feature
            X_permuted = X.copy()
            X_permuted[:, i] = np.random.permutation(X_permuted[:, i])
            
            # Calculate accuracy with permuted feature
            permuted_predictions = model.predict(X_permuted)
            permuted_accuracy = np.mean(permuted_predictions == y)
            
            # Importance = drop in accuracy
            importance = baseline_accuracy - permuted_accuracy
            feature_importances.append(importance)
        
        # Average over repeats
        importances.append(np.mean(feature_importances))
    
    return np.array(importances)

def weight_based_importance(model, feature_names):
    """
    Calculate feature importance based on weights of first layer
    """
    # Sum of absolute weights connecting each input to hidden layer
    importances = np.sum(np.abs(model.W1), axis=1)
    
    # Normalize to [0, 1]
    importances = importances / np.sum(importances)
    
    return importances

# Feature names
feature_names = ['age', 'credit_score', 'balance', 'products_number', 
                'estimated_salary', 'country', 'gender', 'tenure', 
                'has_credit_card', 'is_active_member']

print("=" * 60)
print("Feature Importance Analysis")
print("=" * 60)

# Use the previously trained model
print("\n--- Method 1: Permutation Importance ---")
print("Calculating permutation importance (this may take a moment)...")
perm_importance = permutation_importance(model, X_test, y_test, feature_names, n_repeats=10)

print("\n--- Method 2: Weight-Based Importance ---")
weight_importance = weight_based_importance(model, feature_names)

# Combine results
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Permutation_Importance': perm_importance,
    'Weight_Importance': weight_importance
})

# Sort by permutation importance
importance_df = importance_df.sort_values('Permutation_Importance', ascending=False)

print("\n" + "=" * 70)
print("FEATURE IMPORTANCE RESULTS")
print("=" * 70)
print(importance_df.to_string(index=False))

# Visualize importance
print("\n--- Visualizing Feature Importance ---")
plt.figure(figsize=(16, 10))

# Plot 1: Permutation Importance (horizontal bar)
plt.subplot(2, 3, 1)
sorted_idx = np.argsort(perm_importance)
plt.barh(np.array(feature_names)[sorted_idx], perm_importance[sorted_idx], color='steelblue')
plt.xlabel('Importance (Accuracy Drop)')
plt.title('Permutation Feature Importance')
plt.grid(True, alpha=0.3, axis='x')

# Plot 2: Weight-Based Importance (horizontal bar)
plt.subplot(2, 3, 2)
sorted_idx = np.argsort(weight_importance)
plt.barh(np.array(feature_names)[sorted_idx], weight_importance[sorted_idx], color='coral')
plt.xlabel('Normalized Weight')
plt.title('Weight-Based Feature Importance')
plt.grid(True, alpha=0.3, axis='x')

# Plot 3: Comparison of both methods
plt.subplot(2, 3, 3)
x = np.arange(len(feature_names))
width = 0.35

# Normalize permutation importance for comparison
perm_norm = perm_importance / np.max(perm_importance) if np.max(perm_importance) > 0 else perm_importance

sorted_features = importance_df['Feature'].values
sorted_perm = importance_df['Permutation_Importance'].values / np.max(importance_df['Permutation_Importance'].values)
sorted_weight = importance_df['Weight_Importance'].values

x_pos = np.arange(len(sorted_features))
plt.barh(x_pos - width/2, sorted_perm, width, label='Permutation', alpha=0.8)
plt.barh(x_pos + width/2, sorted_weight, width, label='Weight-Based', alpha=0.8)
plt.yticks(x_pos, sorted_features)
plt.xlabel('Normalized Importance')
plt.title('Feature Importance Comparison')
plt.legend()
plt.grid(True, alpha=0.3, axis='x')

# Plot 4: Top 5 most important features
plt.subplot(2, 3, 4)
top_5 = importance_df.head(5)
plt.bar(range(5), top_5['Permutation_Importance'], color='green', alpha=0.7)
plt.xticks(range(5), top_5['Feature'], rotation=45, ha='right')
plt.ylabel('Importance')
plt.title('Top 5 Most Important Features')
plt.grid(True, alpha=0.3, axis='y')

# Plot 5: Correlation between two methods
plt.subplot(2, 3, 5)
plt.scatter(importance_df['Permutation_Importance'], 
           importance_df['Weight_Importance'], 
           s=100, alpha=0.6, color='purple')

for i, feature in enumerate(importance_df['Feature']):
    plt.annotate(feature, 
                (importance_df['Permutation_Importance'].iloc[i], 
                 importance_df['Weight_Importance'].iloc[i]),
                fontsize=8, alpha=0.7)

plt.xlabel('Permutation Importance')
plt.ylabel('Weight-Based Importance')
plt.title('Correlation Between Methods')
plt.grid(True, alpha=0.3)

# Plot 6: Feature importance heatmap
plt.subplot(2, 3, 6)
importance_matrix = importance_df[['Permutation_Importance', 'Weight_Importance']].T.values

# Normalize each row
importance_matrix_norm = importance_matrix / importance_matrix.max(axis=1, keepdims=True)

im = plt.imshow(importance_matrix_norm, cmap='YlOrRd', aspect='auto')
plt.colorbar(im, label='Normalized Importance')
plt.yticks([0, 1], ['Permutation', 'Weight-Based'])
plt.xticks(range(len(feature_names)), importance_df['Feature'], rotation=45, ha='right')
plt.title('Feature Importance Heatmap')

plt.tight_layout()
plt.show()

# Detailed analysis
print("\n" + "=" * 70)
print("DETAILED ANALYSIS")
print("=" * 70)

# Top 3 features
top_3 = importance_df.head(3)
print("\n🏆 Top 3 Most Important Features:")
for idx, row in top_3.iterrows():
    print(f"  {row['Feature']:20s} - Permutation: {row['Permutation_Importance']:.4f}, "
          f"Weight: {row['Weight_Importance']:.4f}")

# Least important features
bottom_3 = importance_df.tail(3)
print("\n⚠ Least Important Features (consider removing):")
for idx, row in bottom_3.iterrows():
    print(f"  {row['Feature']:20s} - Permutation: {row['Permutation_Importance']:.4f}, "
          f"Weight: {row['Weight_Importance']:.4f}")

# Calculate feature importance groups
high_importance = importance_df[importance_df['Permutation_Importance'] > 0.01]
medium_importance = importance_df[(importance_df['Permutation_Importance'] > 0.005) & 
                                 (importance_df['Permutation_Importance'] <= 0.01)]
low_importance = importance_df[importance_df['Permutation_Importance'] <= 0.005]

print("\n" + "=" * 70)
print("FEATURE GROUPING")
print("=" * 70)
print(f"High Importance (>1% accuracy drop): {len(high_importance)} features")
if len(high_importance) > 0:
    print(f"  {', '.join(high_importance['Feature'].values)}")

print(f"\nMedium Importance (0.5-1% accuracy drop): {len(medium_importance)} features")
if len(medium_importance) > 0:
    print(f"  {', '.join(medium_importance['Feature'].values)}")

print(f"\nLow Importance (<0.5% accuracy drop): {len(low_importance)} features")
if len(low_importance) > 0:
    print(f"  {', '.join(low_importance['Feature'].values)}")

# Recommendations
print("\n" + "=" * 70)
print("RECOMMENDATIONS")
print("=" * 70)
print("✓ Focus on high-importance features for business decisions")
print("✓ Consider removing low-importance features to simplify the model")
print("✓ Collect more data for high-importance features if possible")
if len(low_importance) > 0:
    print(f"✓ Potential feature reduction: remove {', '.join(low_importance['Feature'].values)}")